In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import date
import re
import os

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.daraz.com.np/laptops/")

In [ ]:
links = []
for i in range(0, 20):
    products = driver.find_elements(By.XPATH, '//*[@id="id-a-link"]')
    urls = [x.get_property("href") for x in products]
    links += urls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")
    next = driver.find_element(
        By.XPATH,
        '//*[@id="root"]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]/a',
    )
    next.click()
    time.sleep(1)

In [ ]:
df = pd.DataFrame(links, columns=['Url'])
df.to_csv('../datasets/laptops_url.csv', index=False)
link = df["Url"]

In [ ]:
for x in range(0,800):
    print(link[x])
    driver.get(link[x])
    try:
        try:
            actual_price = driver.find_element(
                By.XPATH, '//*[@id="module_product_price_1"]/div/div/div/span[1]'
            ).text

            discount_price = driver.find_element(
                By.XPATH, '//*[@id="module_product_price_1"]/div/div/span'
            ).text
        except:
            actual_price = driver.find_element(
                By.XPATH, '//*[@id="module_product_price_1"]/div/div/span'
            ).text

            discount_price = actual_price

        today = date.today()
        df = pd.DataFrame([[today, actual_price, discount_price]], columns=["Date", "Actual Price", "Discount Price"])
        df["Actual Price"] = df["Actual Price"].str.replace("Rs. ", "").str.replace(",", "")
        df["Discount Price"] = df["Discount Price"].str.replace("Rs. ", "").str.replace(",", "")
        name = str(link[x])
        name = re.sub(r'[^a-zA-Z0-9]', '', name)
        name = re.sub(r'httpswwwdarazcomnpproducts', '', name)
        filename = "../datasets/prices/" + name + ".csv"
        df.to_csv(filename, mode='a', header = not os.path.exists(filename), index=False)
    except:
        print("out of stock")
    time.sleep(1)